# Bleaching samples

## update sample type 
- everything is healthy or diseased right now, 
-> change to bleached based sampling notes
-> then crosscheck with colonydata?

In [68]:
import numpy as np
import pandas as pd
import os
import re

In [69]:
os.getcwd() 

'/Users/brookesienkiewicz/Documents/Code_notebook/QA_QC/SampleData'

In [70]:
# upload sample and colony data 
sample=pd.read_csv('~/Documents/sctld/SCTLD_samples/Sample_Data/CBC_samples.csv')
colony=pd.read_csv('~/Documents/sctld/SCTLD_samples/Sample_Data/CBC_ColonyData.csv')
colony.drop(columns="Unnamed: 0",inplace=True)

### Sample Fixes 
- first fix bad formatting/syntax 

In [71]:
# make all species names uniform 
species_list=sample['Species'].unique()
print(species_list)

['OFAV' 'SSID' 'PAST' 'PSTR' 'MCAV' 'OANN' 'DLAB' 'CNAT' 'DL' 'MMEA'
 'PAST ' 'Unknown' 'OFAV ']


In [72]:
# Change OFAV/OANN variations to ORBI
# already did this one 

# sample.loc[:,'Species']=sample['Species'].str.replace('OANN/OFAV?',"ORBI")
# sample.loc[:,'Species']=sample['Species'].str.replace('OFAV/OANN',"ORBI")

# Fix space in PAST in OFAV
sample.loc[:,'Species']=sample['Species'].str.replace('PAST ',"PAST")
sample.loc[:,'Species']=sample['Species'].str.replace('OFAV ',"OFAV")

# Change 'DL' -> DLAB 
sample['Species'] = sample['Species'].replace('\\bDL\\b', 'DLAB', regex=True)

In [73]:
# Check that changes worked and store specie names in list
species_list=sample['Species'].unique()
print(species_list)

['OFAV' 'SSID' 'PAST' 'PSTR' 'MCAV' 'OANN' 'DLAB' 'CNAT' 'MMEA' 'Unknown']


### Add Bleaching sample type 

In [74]:
# search for sampling notes with clp or clb 

#if col contains clb or clp 
# replace status with bleached tissue 
sample.loc[
    sample['Sampling_notes'].str.contains('CLP|CLB', case=False, na=False),'Health_status']= 'Bleached_Tissue'
# (code is in place replacement)

In [75]:
# match sample type to colony statuses
# got from my cbc_metagenomics code

In [76]:
# add colony ID - t# newtagnum species
sample = sample.copy()
sample['TransectNum_str'] = 'T' + sample['TransectNum'].astype(str)
sample['colony_id'] = sample[['TransectNum_str', 'NewTagNum', 'Species']].astype(str).agg('_'.join, axis=1)
sample.drop(columns='TransectNum_str', inplace=True)

In [77]:
# add colony ID - t# newtagnum species
colony = colony.copy()
colony['TransectNum_str'] = 'T' + colony['TransectNum'].astype(str)
colony['colony_id'] = colony[['TransectNum_str', 'NewTagNum', 'Species']].astype(str).agg('_'.join, axis=1)
colony.drop(columns='TransectNum_str', inplace=True)

In [78]:
# filter for UML samples
sample_subset=sample[
    (sample['Sample_type'] == 'Core_EtOH') |
    (sample['Sample_type'] == 'Core_RNAlater')
]

In [79]:
## BELOW SCRIPT IS IN PROGRESS 8/20/2025 

In [80]:
# extract condition columns 
cond_cols=['colony_id'] + colony.columns[colony.columns.str.contains('_Condition')].tolist()

In [90]:
# make merged df for everything

# filter colony data for conditions 
small_colony=colony.loc[:,cond_cols]

# and pivot
conditions_long=small_colony.melt(id_vars='colony_id', 
                          var_name='Month_year', 
                          value_name='colony_condition')

# extract month_year from column names to match sample data format
conditions_long['Month_year'] = conditions_long['Month_year'].str.extract(r'(\d+)').astype(int)
# add back in colony data 
cols_keep=['colony_id','Date_InitialTag','Transect','Species','immune_y/n','Date_DocumentedMortality']
colony_filtered=conditions_long.merge(colony[cols_keep],
                                      on='colony_id', 
                                      how='left')

# filter sample data (USING SUBSET FOR NOW) 
filtered_samples=sample_subset.loc[:,('colony_id','Month_year','Health_status','Sampling_notes','Tubelabel_species')] 
# rename column for sample health statuses 
filtered_samples = filtered_samples.rename(columns={'Health_status': 'sample_condition'})

# merge colony and sample dfs by colony id and monthyear
merged = pd.merge(
    colony_filtered,
    filtered_samples,
    on=['colony_id', 'Month_year'],
    how='outer'
)

In [91]:
merged.loc[merged['colony_condition']=='CLP',:].head(5)

,colony_id,Month_year,colony_condition,Date_InitialTag,Transect,Species,immune_y/n,Date_DocumentedMortality,sample_condition,Sampling_notes,Tubelabel_species
41,T1_12_PSTR,112023,CLP,10/15/19,CBC30N,PSTR,y,Healthy,Bleached_Tissue,100% CLP,112023_BEL_CBC_T1_278_PSTR
54,T1_13_PAST,112023,CLP,6/24/19,CBC30N,PAST,y,12/16/23,NaN,NaN,NaN
152,T1_20_PAST,12024,CLP,6/24/19,CBC30N,PAST,n,4/26/24,NaN,NaN,NaN
166,T1_21_PAST,12024,CLP,6/24/19,CBC30N,PAST,y,8/1/24,Healthy,NaN,012024_BEL_CBC_T1_565_PAST
174,T1_21_PAST,92023,CLP,6/24/19,CBC30N,PAST,y,8/1/24,Healthy,No CL,092023_BEL_CBC_T1_172_PAST


In [92]:
# make merged df for everything (not just by species) and use code below 

# if sample id is not na, colony condition is CLP/CLB and sample status is healthy, 
# replace sample status to bleached_tissue 
# then somehow replace original sample data? maybe just by matching tubelabel?

bl_conds = ['CLP','CLB','CLP,CLB','CLB,CLP']
# get rows with no samples 
# rows_na = (merged['sample_ids'].explode().isna().groupby(level=0).any())

# get rows with clp/clb conditions 
rows = (merged['colony_condition'].isin(bl_conds) &
        # exclude rows with no samples 
        merged['Tubelabel_species'].notna())  

# update only those rows with bleached sample status 
merged.loc[rows, 'sample_condition'] = 'Bleached_Tissue'

In [93]:
merged.loc[merged['colony_condition']=='CLP',:].head(5)
# check it worked

,colony_id,Month_year,colony_condition,Date_InitialTag,Transect,Species,immune_y/n,Date_DocumentedMortality,sample_condition,Sampling_notes,Tubelabel_species
41,T1_12_PSTR,112023,CLP,10/15/19,CBC30N,PSTR,y,Healthy,Bleached_Tissue,100% CLP,112023_BEL_CBC_T1_278_PSTR
54,T1_13_PAST,112023,CLP,6/24/19,CBC30N,PAST,y,12/16/23,NaN,NaN,NaN
152,T1_20_PAST,12024,CLP,6/24/19,CBC30N,PAST,n,4/26/24,NaN,NaN,NaN
166,T1_21_PAST,12024,CLP,6/24/19,CBC30N,PAST,y,8/1/24,Bleached_Tissue,NaN,012024_BEL_CBC_T1_565_PAST
174,T1_21_PAST,92023,CLP,6/24/19,CBC30N,PAST,y,8/1/24,Bleached_Tissue,No CL,092023_BEL_CBC_T1_172_PAST


In [94]:
# create function to crosscheck colony health with samples 

def status_match(group):
    # get the condition and list sample statuses of each colony at each monthyear (by row) 
    cond = str(group['colony_condition'].iloc[0])
    # turned into string to prevent crashes from NAs
    statuses = group['sample_condition'].dropna().tolist()
    month = group['Month_year'].iloc[0]
    immune_dates = [112023, 122023, 12024, 22024, 62024, 82024, 122024] 

    # match colony conditions to sample conditions 
    # if colony_condition is healthy, sample = healthy 
    if cond == 'Healthy':
        return all(s == 'Healthy' for s in statuses)

    # if condition is diseased healthy = diseased_margin and diseased_tissue (but rn these are on 2 diff rows)
    elif cond in ['Diseased','Diseased_Other']:
        return all(x in statuses for x in ['Diseased_Tissue', 'Diseased_Margin'])

    # if condition is clp | clb = bleached_tissue 
    elif any(term in cond for term in ['CLP','CLB']):
        return all(x in statuses for x in ['Bleached_Tissue'])
    
    # if condition is Not_visited, sample = NaN
    # if condition is Dead, sample = NaN 
    elif cond in ['Dead', 'Not_Visited','Not_visited','not_visited','NAN']:
        return all(pd.isna(s) for s in group['sample_condition'])

    # if condition is na, sample should also be na 
    elif pd.isna(cond):
        return all(pd.isna(s) for s in group['sample_condition'])

    # exclude False matches if the colony is non-immune during an immune sampling point 
    # list of immune dates 
    
    # if coral is NOT immune during an immune date 
    elif group['immune_y/n'].iloc[0]=='n' and month in immune_dates:
        # it is ok if the sample status is na 
        return all(pd.isna(s) for s in group['sample_condition'])

    ## UGH but this has to exclude T5 and T6 during 01/2024 and 042024 respectively because they were newly tagged then 

    # any mismatches return as false 
    else:
        return False

In [95]:
merged.groupby(['colony_id', 'Month_year'])

In [107]:
# crosscheck colony and sample conditions

checks = []
    # check each 'group' (unique combos of colony and monthyear) and store results in 'checks' list
for (colony_id, month), group in merged.groupby(['colony_id', 'Month_year']):
    checks.append({
        'colony_id': colony_id,
        'Species': group['Species'].iloc[0],
        'immune_y/n': group['immune_y/n'].iloc[0],
        'Date_InitialTag': group['Date_InitialTag'].iloc[0],
        'Month_year': month,
            
        # run function &
        # create col containing results of the function that matches statuses
        'Match': status_match(group),
            
        'colony_condition': group['colony_condition'].iloc[0],
        'sample_condition': group['sample_condition'].tolist(),
        'sample_ids': group['Tubelabel_species'].tolist(),
        'mortality_date' : group['Date_DocumentedMortality'].iloc[0]
})
# convert to dataframe
checks_df=pd.DataFrame(checks)

In [ ]:

checks_df[checks_df['Match']==False]

In [109]:
# check mismatches 

false_matches = {}
for specie in species_list:
    # make individual dfs per sp 
    df = checks_df[checks_df['Species']==specie]
    df.head(2)

    # exclude 122024 since no samples were taken 
    df = df[df['Month_year'] != 122024]
    
    # filter for where the match is false or if the colony condition is na and create a new df 
    false = df[(df['Match'] == False) & (~df['colony_condition'].isna())]
    
    # store false matches in new dictionary with species name as key 
    false_matches[specie] = false

In [110]:
false_matches['MCAV']

,colony_id,Species,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_condition,sample_ids,mortality_date
459,T1_8_MCAV,MCAV,n,6/24/19,122022,False,Diseased,[Diseased_Tissue],[122022_BEL_CBC_T1_144_MCAV],9/25/23
717,T2_53_MCAV,MCAV,n,6/22/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1019_MCAV],Healthy
744,T2_55_MCAV,MCAV,y,6/25/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1016_MCAV],Healthy
812,T2_59_MCAV,MCAV,n,6/21/19,52022,False,Diseased,[nan],[nan],Diseased
949,T2_69_MCAV,MCAV,y,6/25/19,92023,False,Healthy,[Bleached_Tissue],[092023_BEL_CBC_T2_176_MCAV],Healthy
1184,T3_14_MCAV,MCAV,y,6/23/19,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T3_1711_MCAV],Healthy
1375,T3_2_MCAV,MCAV,y,6/23/19,12024,False,"Diseased_Other, CLP",[Healthy],[012024_BEL_CBC_T3_633_MCAV],Diseased
1376,T3_2_MCAV,MCAV,y,6/23/19,22024,False,Diseased_Other,"[Bleached_Tissue, Bleached_Tissue]","[022024_BEL_CBC_T3_842_MCAV, 022024_BEL_CBC_T3...",Diseased
1377,T3_2_MCAV,MCAV,y,6/23/19,42024,False,Diseased_Other,[Healthy],[042024_BEL_CBC_T3_956_MCAV],Diseased
1382,T3_2_MCAV,MCAV,y,6/23/19,82024,False,Diseased_Other,[Healthy],[082024_BEL_CBC_T3_1553_MCAV],Diseased


In [ ]:
# create for loop for each specie - using the fxn to match the colony and sample health statuses 
species_checks = {}

for specie in species_list: 
    # filter colony and sample data for each specie 
    filtered_colony=colony[colony['Species']==specie]

    # extract condition columns 
    small_colony= filtered_colony.loc[:,cond_cols]
    # and pivot
    conditions_long=small_colony.melt(id_vars='colony_id', 
                              var_name='Month_year', 
                              value_name='colony_condition')
    
    # extract month_year from column names to match sample data 
    conditions_long['Month_year'] = conditions_long['Month_year'].str.extract(r'(\d+)').astype(int)
    print(conditions_long)
    # repeat for sample data 
    filtered_samples=sample_subset[sample_subset['Species']==specie]
    filtered_samples=filtered_samples.loc[:,('colony_id','Month_year','Health_status','Sampling_notes','Tubelabel_species')] 
    # rename column for sample health statuses 
    filtered_samples['sample_condition']=filtered_samples['Health_status']
    
    # merge dfs by colony id and monthyear
    merged = pd.merge(
    conditions_long.merge(filtered_colony, on='colony_id', how='left'),
    filtered_samples,
    on=['colony_id', 'Month_year'],
    how='outer'
    )
    print(merged)
    checks = []
        # check each 'group' (unique combos of colony and monthyear) and store results in 'checks' list
    for (colony_id, month), group in merged.groupby(['colony_id', 'Month_year']):
        checks.append({
            'colony_id': colony_id,
            'immune_y/n': group['immune_y/n'].iloc[0],
            'Date_InitialTag': group['Date_InitialTag'].iloc[0],
            'Month_year': month,
                
            # run function &
            # create col containing results of the function that matches statuses
            'Match': status_match(group),
                
            'colony_condition': group['colony_condition'].iloc[0],
            'sample_statuses': group['sample_condition'].tolist(),
            'sample_ids': group['Tubelabel_species'].tolist(),
            'mortality_date' : group['Date_DocumentedMortality'].iloc[0]
    })
    # convert to dataframe and store each species df in a dictionary 'species_checks'
    species_checks[specie]=pd.DataFrame(checks)
    

In [20]:
species_checks['MCAV']['Month_year'].dtype

dtype('int64')

In [21]:
# check mismatches 
false_matches = {}
for specie in species_list:
    # make individual dfs per sp 
    df = pd.DataFrame(species_checks[specie])

    # exclude 122024 since no samples were taken 
    df = df[df['Month_year'] != 122024]
    
    # filter for where the match is false or if the colony condition is na and create a new df 
    false = df[(df['Match'] == False) & (~df['colony_condition'].isna())]
    
    # store false matches in new dictionary with species name as key 
    false_matches[specie] = false

In [22]:
species_list

array(['OFAV', 'SSID', 'PAST', 'PSTR', 'MCAV', 'OANN', 'DLAB', 'CNAT',
       'MMEA', 'Unknown'], dtype=object)

In [23]:
# view mismatches for each species 
test=false_matches['MCAV']
test

,colony_id,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
99,T1_7_MCAV,n,6/24/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T1_181_MCAV],12/1/24
114,T1_8_MCAV,n,6/24/19,122022,False,Diseased,[Diseased_Tissue],[122022_BEL_CBC_T1_144_MCAV],9/25/23
119,T2_53_MCAV,n,6/22/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1019_MCAV],Healthy
130,T2_55_MCAV,y,6/25/19,12024,False,CLP,[Healthy],[012024_BEL_CBC_T2_603_MCAV],Healthy
132,T2_55_MCAV,y,6/25/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1016_MCAV],Healthy
159,T2_59_MCAV,n,6/21/19,52022,False,Diseased,[nan],[nan],Diseased
164,T2_59_MCAV,n,6/21/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T2_220_MCAV],Diseased
180,T2_60_MCAV,y,6/22/19,122023,False,CLB,[Healthy],[122023_BEL_CBC_T2_522_MCAV],Healthy
203,T2_69_MCAV,y,6/25/19,92023,False,Healthy,[Bleached_Tissue],[092023_BEL_CBC_T2_176_MCAV],Healthy
227,T3_14_MCAV,y,6/23/19,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T3_1711_MCAV],Healthy


In [33]:
# test on TEST first 
mask_any_na = test['sample_ids'].explode().isna().groupby(level=0).any()
test[mask_any_na]

,colony_id,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
159,T2_59_MCAV,n,6/21/19,52022,False,Diseased,[nan],[nan],Diseased


In [44]:
# if sample id is not na, colony condition is CLP/CLB and sample status is healthy, 
# replace sample status to bleached_tissue 
# then somehow replace original sample data? maybe just by matching tubelabel?

bl_conds = ['CLP','CLB','CLP,CLB','CLB,CLP']

# make copy 
updated = test.copy()

# get rows with no samples 
rows_na = (updated['sample_ids'].explode().isna().groupby(level=0).any())

# get rows with clp/clb conditions 
rows = (updated['colony_condition'].isin(bl_conds) &
        # exclude rows with no samples 
        ~rows_na)  

# update only those rows with bleached sample status 
updated.loc[rows, 'sample_statuses'] = 'Bleached_Tissue'

In [45]:
updated

,colony_id,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
99,T1_7_MCAV,n,6/24/19,92023,False,CLP,Bleached_Tissue,[092023_BEL_CBC_T1_181_MCAV],12/1/24
114,T1_8_MCAV,n,6/24/19,122022,False,Diseased,[Diseased_Tissue],[122022_BEL_CBC_T1_144_MCAV],9/25/23
119,T2_53_MCAV,n,6/22/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1019_MCAV],Healthy
130,T2_55_MCAV,y,6/25/19,12024,False,CLP,Bleached_Tissue,[012024_BEL_CBC_T2_603_MCAV],Healthy
132,T2_55_MCAV,y,6/25/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1016_MCAV],Healthy
159,T2_59_MCAV,n,6/21/19,52022,False,Diseased,[nan],[nan],Diseased
164,T2_59_MCAV,n,6/21/19,92023,False,CLP,Bleached_Tissue,[092023_BEL_CBC_T2_220_MCAV],Diseased
180,T2_60_MCAV,y,6/22/19,122023,False,CLB,Bleached_Tissue,[122023_BEL_CBC_T2_522_MCAV],Healthy
203,T2_69_MCAV,y,6/25/19,92023,False,Healthy,[Bleached_Tissue],[092023_BEL_CBC_T2_176_MCAV],Healthy
227,T3_14_MCAV,y,6/23/19,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T3_1711_MCAV],Healthy


In [314]:
false_matches['PAST']

,colony_id,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
9,T1_13_PAST,y,6/24/19,112023,False,CLP,[nan],[nan],12/16/23
13,T1_19_PAST,n,6/21/19,12024,False,CLB,[nan],[nan],4/1/24
16,T1_19_PAST,n,6/21/19,52022,False,Diseased,[Diseased_Tissue],[052022_BEL_CBC_T1_52_PAST],4/1/24
21,T1_19_PAST,n,6/21/19,92023,False,CLB,[nan],[nan],4/1/24
26,T1_20_PAST,n,6/24/19,12024,False,CLP,[nan],[nan],4/26/24
39,T1_21_PAST,y,6/24/19,12024,False,CLP,[Healthy],[012024_BEL_CBC_T1_565_PAST],8/1/24
47,T1_21_PAST,y,6/24/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T1_172_PAST],8/1/24
48,T1_21_PAST,y,6/24/19,112023,False,CLP,[Healthy],[112023_BEL_CBC_T1_274_PAST],8/1/24
50,T1_21_PAST,y,6/24/19,122023,False,"CLB,CLP",[Healthy],[122023_BEL_CBC_T1_507_PAST],8/1/24
52,T1_2_PAST,y,6/21/19,12024,False,"CLP,CLB",[nan],[nan],Healthy


In [315]:
false_matches['SSID']
# why do we have some that say not visited but we have samples for..

,colony_id,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
13,T1_11_SSID,n,6/24/19,12024,False,Diseased,[nan],[nan],Diseased
15,T1_11_SSID,n,6/24/19,42024,False,Diseased,[Diseased_Margin],[042024_BEL_CBC_T1_936_SSID],Diseased
21,T1_11_SSID,n,6/24/19,92023,False,Diseased,[Diseased_Margin],[092023_BEL_CBC_T1_193_SSID],Diseased
26,T1_16_SSID,n,6/24/19,12024,False,Diseased,[nan],[nan],Diseased
32,T1_16_SSID,n,6/24/19,62025,False,Diseased,[nan],[nan],Diseased
...,...,...,...,...,...,...,...,...,...
333,T3_36_SSID,n,6/23/19,92023,False,DC,[Healthy],[092023_BEL_CBC_T3_190_SSID],Diseased
338,T3_3_SSID,n,6/23/19,12024,False,Diseased,[nan],[nan],Diseased
358,T3_5_SSID,y,6/23/19,82024,False,CLP,[Healthy],[082024_BEL_CBC_T3_1555_SSID],Healthy
366,T3_65_SSID,n,6/23/19,42024,False,Not_Visited,[Diseased_Margin],[042024_BEL_CBC_T3_964_SSID],Diseased


In [316]:
false_matches['PSTR']

,colony_id,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
10,T1_12_PSTR,y,10/15/19,112023,False,CLP,[Healthy],[112023_BEL_CBC_T1_278_PSTR],Healthy
119,T2_32_PSTR,n,12/5/22,92023,False,CLP,[Healthy],[092023_BEL_CBC_T2_197_PSTR],Healthy
152,T2_54_PSTR,n,10/13/19,12024,False,CLP,[nan],[nan],Healthy
160,T2_54_PSTR,n,10/13/19,92023,False,"Diseased, CLP","[Diseased_Margin, Diseased_Tissue]","[092023_BEL_CBC_T2_185_PSTR, 092023_BEL_CBC_T2...",Healthy
258,T3_20_PSTR,y,10/14/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T3_172_PSTR],Healthy
260,T3_20_PSTR,y,10/14/19,112023,False,CLP,[Healthy],[112023_BEL_CBC_T3_354_PSTR],Healthy
314,T3_4_PSTR,n,10/14/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T3_185_PSTR],Healthy
320,T3_70_PSTR,y,12/3/22,12024,False,CLP,[Healthy],[012024_BEL_CBC_T3_644_PSTR],Healthy
326,T3_70_PSTR,y,12/3/22,62025,False,CLP,[CLP],[062025_BEL_CBC_T3_1685_PSTR],Healthy
328,T3_70_PSTR,y,12/3/22,92023,False,CLP,[Healthy],[092023_BEL_CBC_T3_195_PSTR],Healthy


In [317]:
# if the colony condition says clb or clp, make the sample status bleached_tissue 
#make list of colony ids that have clb or clp in colony condition and healthy as sample status 

false_matches
for specie in species_list:
    # filter dict for colony ids that have clb or clp in colony condition and healthy as sample status 
    false_matches[specie]